In [2]:
import chess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from tqdm import tqdm

import cairosvg
from PIL import Image
from PIL import ImageFilter
from io import BytesIO

from itertools import count

In [3]:
df = pd.read_csv('results/2024-02-16_results.csv')

hist = df.history.values.tolist()
hist = [x.split("\t") for x in hist]
hist = hist[np.random.randint(0, len(hist))]

hist[0][:3]

'rnb'

In [ ]:
w = 16
h = 9
n_frames = 100
scales = np.linspace(0.12, 1, n_frames)

fig, ax = plt.subplots(1, 1, figsize=(w, h))

fig.set_facecolor('black')
fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

for t, scale in tqdm(zip(count(), scales), total=n_frames):
    ax.clear()
    ax.axis('off')
    ax.set_facecolor('black')

    x0 = w/2 * (1 - scale)
    x1 = w/2 * (1 + scale)
    y0 = h/2 * (1 - scale)
    y1 = h/2 * (1 + scale)

    ax.set_xlim(x0, x1)
    ax.set_ylim(y0, y1)

    for i in range(w * h):
        boundaries = [(i % w), (i % w + 1), (i // w), (i // w + 1)]

        if boundaries[1] < x0 or boundaries[0] > x1 or boundaries[3] < y0 or boundaries[2] > y1:
            continue

        game = hist[i % len(hist)]
        board_fen = game[t % len(game)]
        board = chess.Board(board_fen)
        svg = chess.svg.board(board=board)
        img = Image.open(BytesIO(cairosvg.svg2png(bytestring=svg, )))

        ax.imshow(img, aspect="equal", extent=boundaries)

    # ax.plot([x0, x1], [y0, y0], 'r')
    # ax.plot([x0, x1], [y1, y1], 'r')
    # ax.plot([x0, x0], [y0, y1], 'r')
    # ax.plot([x1, x1], [y0, y1], 'r')
    
    plt.savefig(f'frames/frame_{t:04d}.png')

In [12]:
# for every frame, put the CBT.png image on top of the board

new_dir = "./new_frames"

imgs = [f'frames/frame_{t:04d}.png' for t in range(n_frames)]

space = np.linspace(0, 1, n_frames)

for t in tqdm(range(n_frames)):
    img = Image.open(imgs[t])

    # blur image
    img = img.filter(ImageFilter.GaussianBlur(radius=space[t] * 2))

    # darken image
    img = img.point(lambda p: (- space[t] * 0.8 + 1) * p )

    cbt = Image.open("CBT.png")
    img.paste(cbt, (0, 0, img.width, img.height), cbt)
    img.save(f"{new_dir}/frame_{t:04d}.png")

100%|██████████| 100/100 [00:54<00:00,  1.84it/s]


In [13]:
# make a gif

import imageio
import os

images = []
frame_names = os.listdir('new_frames')
frame_names.sort()
for filename in frame_names:
    images.append(imageio.imread(f'new_frames/{filename}'))

imageio.mimsave('chess.gif', images, duration=3)

/var/folders/w3/2rfcg8md39x8h72m0ssfy9n80000gn/T/ipykernel_41821/3831224420.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f'new_frames/{filename}'))
